# **ETL Metadata-sitios**
---

In [ ]:
#Librerias a utilizar
from google.colab import drive
import pandas as pd
import json
import pyarrow.parquet as pq


# Establece conexión con google Drive
drive.mount('/content/drive')



def aplicar_ETL():
  # Variables donde se almacenan las rutas de los archivos crudos
  metadata_9 ='/content/drive/MyDrive/Proyecto Final/Google Maps/metadata-sitios/9.json'
  metadata_11 ='/content/drive/MyDrive/Proyecto Final/Google Maps/metadata-sitios/11.json'
  metadata_10 ='/content/drive/MyDrive/Proyecto Final/Google Maps/metadata-sitios/10.json'
  metadata_6 ='/content/drive/MyDrive/Proyecto Final/Google Maps/metadata-sitios/6.json'

  rutas=[metadata_6,metadata_9,metadata_10,metadata_11]
  # Desarrollo de ETL---------------------------------
  for ruta in rutas:

    try:
      metadata_final=[]
      #  Lee archivo y devuelve un dataframe de pandas
      metadata=pd.read_json(metadata_9,lines=True)
      # Eliminación de columna importantes
      df_metadata=metadata.drop(columns=['price', 'hours',	'state',	'relative_results', 'url'])
      # Lista de estados (utilizando regex)
      lista_estados= ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
      regex = '|'.join(lista_estados)
      # Crear columna estados a partir de columna "address"
      df_metadata['state']=metadata['address'].str.extract(r'.*?(\b(?:{})\b).*'.format(regex))
      # Filtrar por los estados seleccionados
      estados=['PA','FL','CA','IL']
      df_metadata=df_metadata[df_metadata['state'].isin(estados)].reset_index(drop=True)
      # Elimina valores nulos
      df_metadata=df_metadata.dropna().reset_index(drop=True)
      # Cambio de Tipos
      df_metadata['latitude']=df_metadata['latitude'].astype('float')
      df_metadata['longitude']=df_metadata['longitude'].astype('float')

      # Trabajaremos con los negocios que contengan mas de 20 reseñas, ya que menos reseñas no seria eficiente para el sistema de recomendación
      df_metadata=df_metadata[df_metadata['num_of_reviews']>20].reset_index(drop=True)
      # Adaptar a formato string la columna "category"
      df_metadata['category']=df_metadata['category'].apply(str).str.strip('[').str.strip(']')
      # Filtrado de negocios por rubro gastronómico
      palabras_clave=["Restaurant","Hamburger restaurant","Restaurant, Bar","Bar","Restaurants","Food","Bubble Tea","Coffee & Tea","Bakery","Barbecue","Buffets","Burgers","Cafes","Catering","Cocktail Bars","Desserts","Diners","Food Trucks","Ice Cream & Frozen Yogurt","Juice Bars & Smoothies","Pizza","Seafood","Steakhouses","Sushi Bars","Tapas Bars","Wine Bars","Vegetarian","Vegan","Gluten-Free"]
      # Transformar columna "category" en string
      filtro_gastronomico=df_metadata['category'].str.contains('|'.join(palabras_clave),case=False)
      df_metadata=df_metadata[filtro_gastronomico].reset_index(drop=True)
      metadata_final.append(df_metadata)



    except: pass
  df_final=pd.concat(metadata_final).reset_index(drop=True)
    # Exportar en formato parquet
  df_final.to_parquet('metadata_procesado.parquet')

    # # Muestra datasets ya procesados
  print('\n---------------------------------------------------------------------Dataset procesado-------------------------------------------------------\n')
  display(df_final)

Mounted at /content/drive


In [ ]:
# EJECUTA ETL
aplicar_ETL()

ValueError: No objects to concatenate

In [ ]:
# ADAPTACION DE CODIGO PARA GCP



# Eliminación de columna importantes
datos=datos.drop(columns=['price', 'hours', 'MISC',	'state',	'relative_results', 'url'])
# Lista de estados (utilizando regex)
lista_estados= ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
regex = '|'.join(lista_estados)
# Crear columna estados a partir de columna "address"
datos['state']=datos['address'].str.extract(r'.*?(\b(?:{})\b).*'.format(regex))
# Filtrar por los estados seleccionados
estados=['PA','FL','CA','IL']
datos=datos[datos['state'].isin(estados)].reset_index(drop=True)
datos=datos.explode('category')
# Elimina valores nulos
datos=datos.dropna().reset_index(drop=True)
# Cambio de Tipos
datos['latitude']=datos['latitude'].astype('float')
datos['longitude']=datos['longitude'].astype('float')

# Trabajaremos con los negocios que contengan mas de 20 reseñas, ya que menos reseñas no seria eficiente para el sistema de recomendación
datos=datos[datos['num_of_reviews']>20].reset_index(drop=True)
# Filtrado de negocios por rubro gastronómico
palabras_clave=["Restaurant","Hamburger restaurant","Restaurant, Bar","Bar","Restaurants","Food","Bubble Tea","Coffee & Tea","Bakery","Barbecue","Buffets","Burgers","Cafes","Catering","Cocktail Bars","Desserts","Diners","Food Trucks","Ice Cream & Frozen Yogurt","Juice Bars & Smoothies","Pizza","Seafood","Steakhouses","Sushi Bars","Tapas Bars","Wine Bars","Vegetarian","Vegan","Gluten-Free"]
# Transformar columna "category" en string
filtro_gastronomico=datos['category'].str.contains('|'.join(palabras_clave),case=False)
datos=datos[filtro_gastronomico].reset_index(drop=True)



In [23]:
#Librerias a utilizar
from google.colab import drive
import pandas as pd
import json
import pyarrow.parquet as pq


# Establece conexión con google Drive
drive.mount('/content/drive')
metadata_9 ='/content/drive/MyDrive/Proyecto Final/Google Maps/metadata-sitios/9.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
datos=pd.read_json(metadata_9,lines=True)
# Eliminar nulos de la columna MISC
datos=datos.dropna(subset='MISC').reset_index(drop=True)
dummies=pd.DataFrame(datos['MISC'].tolist())
dummies=dummies.loc[:,['Service options','Accessibility','Planning','Amenities',]]
datos=pd.concat([datos,dummies],axis=1)
# Eliminación de columna importantes
datos=datos.drop(columns=['hours', 'MISC','state',	'relative_results', 'url','description'])
# Lista de estados (utilizando regex)
lista_estados= ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
regex = '|'.join(lista_estados)
# Crear columna estados a partir de columna "address"
datos['state']=datos['address'].str.extract(r'.*?(\b(?:{})\b).*'.format(regex))
# Filtrar por los estados seleccionados
estados=['PA','FL','CA','IL']
datos=datos[datos['state'].isin(estados)].reset_index(drop=True)
# datos=datos.explode('category')
datos

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,price,Service options,Accessibility,Planning,Amenities,state
0,MemorialCare Breast Center - Irvine Sand Canyon,MemorialCare Breast Center - Irvine Sand Canyo...,0x80dcdd3b95c7284b:0x23d9b4440c982d37,33.668389,-117.765649,[Mammography service],4.7,33,None,NaN,[Wheelchair accessible entrance],NaN,NaN,CA
1,Nick's Auto Service,"Nick's Auto Service, 16291 Construction Cir E ...",0x80dcdc81a6d04bd3:0x72d18ce084849c10,33.694544,-117.821692,"[Auto repair shop, Brake shop, Car inspection ...",4.3,78,None,NaN,[Wheelchair accessible entrance],[Appointments recommended],[Mechanic],CA
2,Alton Self Storage,"Alton Self Storage, 2215 Alton Pkwy, Irvine, C...",0x80dcde9ed485d8bb:0xadfae03de02d931d,33.692216,-117.837010,[Self-storage facility],4.3,118,None,NaN,[Wheelchair accessible entrance],NaN,NaN,CA
3,Public Storage,"Public Storage, 17052 Jamboree Rd, Irvine, CA ...",0x80dcde9d3e4c7391:0x6b9de40d358e241,33.687345,-117.831555,"[Self-storage facility, Automobile storage fac...",4.9,388,None,NaN,[Wheelchair accessible entrance],NaN,NaN,CA
4,Stagecoach Inn Restaurant and Lodge,"Stagecoach Inn Restaurant and Lodge, 204 S Bay...",0x87de5daa7cc7aefd:0x4220d709052263db,39.441171,-90.870254,"[Restaurant, Dessert shop, Hamburger restauran...",4.6,38,None,"[Outdoor seating, Curbside pickup, No-contact ...","[Wheelchair accessible elevator, Wheelchair ac...",[Accepts reservations],"[Good for kids, High chairs, Restroom, Wi-Fi]",IL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55040,Wild Jacks Tex-Mex BBQ,"Wild Jacks Tex-Mex BBQ, 33300 Bernard Dr, Kett...",0x80eb4d458b0ca469:0x77714e3fe8ec0d00,35.988110,-119.958199,[Barbecue restaurant],3.7,88,None,"[Outdoor seating, Takeout, Delivery]",[Wheelchair accessible entrance],NaN,[Good for kids],CA
55041,Scarlett O Haira's,"Scarlett O Haira's, 2845 N Military Trl # 27, ...",0x88d929ef98b5ff25:0xa550550941e533f4,26.719270,-80.112359,[Beauty salon],4.8,18,None,NaN,"[Wheelchair accessible entrance, Wheelchair ac...",[Appointments recommended],"[Gender-neutral restroom, Restroom, Wi-Fi]",FL
55042,Jared,"Jared, 11360 Legacy Ave Suite 100, Palm Beach ...",0x88df2aa67c602449:0x8dc3c40d83e9a74a,26.844319,-80.090218,"[Jewelry store, Diamond dealer, Gift shop, Gol...",4.2,58,$$$,"[In-store pickup, In-store shopping, Delivery]",[Wheelchair accessible entrance],NaN,NaN,FL
55043,Whispering Creek Co-Op Inc,"Whispering Creek Co-Op Inc, 2023 St Lucie Blvd...",0x88def137f3d9258f:0xb1dc032eea0cf50f,27.482197,-80.342480,"[Mobile home park, Retirement community]",3.7,68,None,NaN,[Wheelchair accessible entrance],NaN,NaN,FL


In [47]:
# Elimina valores nulos
datos=datos.dropna().reset_index(drop=True)
# Cambio de Tipos
datos['latitude']=datos['latitude'].astype('float')
datos['longitude']=datos['longitude'].astype('float')

In [48]:

# Trabajaremos con los negocios que contengan mas de 20 reseñas, ya que menos reseñas no seria eficiente para el sistema de recomendación
datos=datos[datos['num_of_reviews']>20].reset_index(drop=True)
# Adaptar a formato string la columna "category"
datos['category']=datos['category'].apply(str).str.strip('[').str.strip(']')
# Filtrado de negocios por rubro gastronómico
palabras_clave=["Restaurant","Hamburger restaurant","Restaurant, Bar","Bar","Restaurants","Food","Bubble Tea","Coffee & Tea","Bakery","Barbecue","Buffets","Burgers","Cafes","Catering","Cocktail Bars","Desserts","Diners","Food Trucks","Ice Cream & Frozen Yogurt","Juice Bars & Smoothies","Pizza","Seafood","Steakhouses","Sushi Bars","Tapas Bars","Wine Bars","Vegetarian","Vegan","Gluten-Free"]
# Transformar columna "category" en string
filtro_gastronomico=datos['category'].str.contains('|'.join(palabras_clave),case=False)
datos=datos[filtro_gastronomico].reset_index(drop=True)
datos

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,price,Service options,Accessibility,Planning,Amenities,state
0,Happy Garden,"Happy Garden, 605 Main St, Chester, CA 96020",0x809d96339c446c6f:0x2e264c7b998a5962,40.299243,-121.237057,'Chinese restaurant',4.5,176,$$,"[Takeout, Dine-in, Delivery]",[Wheelchair accessible entrance],[Accepts reservations],"[Good for kids, High chairs]",CA
1,El Aji,"El Aji, 3015 Mission St, San Francisco, CA 94110",0x808f7e46a851d7c3:0xf3917bd31fb16c42,37.748831,-122.417958,"'Peruvian restaurant', 'Restaurant', 'Seafood ...",4.3,156,$$,"[No-contact delivery, Delivery, Takeout, Dine-in]",[Wheelchair accessible entrance],[Accepts reservations],[Good for kids],CA
2,Rich's,"Rich's, 1051 University Ave, San Diego, CA 92103",0x80d954dcc5ff6143:0x77275aea20662789,32.748263,-117.154034,"'Gay night club', 'Gay bar'",3.8,128,$$,"[Outdoor seating, Dine-in, Delivery, Takeout]","[Wheelchair accessible entrance, Wheelchair ac...",[Accepts reservations],"[Restroom, Wi-Fi]",CA
3,La Quinta Real,"La Quinta Real, 400 E Compton Blvd #3207, Comp...",0x80c2cb6f43cde693:0x74db942c93cedbaf,33.895847,-118.218536,'Restaurant',4.3,154,$$,"[Takeout, Dine-in]",[Wheelchair accessible entrance],"[Dinner reservations recommended, Accepts rese...",[Bar onsite],CA
4,Frida's Mexican Taqueria,"Frida's Mexican Taqueria, 133 N Main St, Telfo...",0x89c6a1d43c88ff77:0x8454344e133e269a,40.325998,-75.329533,'Mexican restaurant',4.9,55,$,"[Takeout, Dine-in]",[Wheelchair accessible entrance],[Accepts reservations],[Good for kids],PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,Starbucks,"Starbucks, 4050 N Harlem Ave, Norridge, IL 60706",0x880fcbb6de68c833:0xfce967f4602cde71,41.954483,-87.808550,"'Coffee shop', 'Cafe', 'Coffee store', 'Espres...",3.2,115,$$,"[Delivery, Takeout, Dine-in]","[Wheelchair accessible entrance, Wheelchair ac...",[Usually a wait],[Restroom],IL
361,CAVA,"CAVA, 44400 Indian Wells Ln, Indian Wells, CA ...",0x80dafeb5db201695:0x26c5725954eec643,33.722850,-116.325226,"'Restaurant', 'American restaurant'",4.1,38,$$,"[Outdoor seating, Takeout, Dine-in, Delivery]","[Wheelchair accessible entrance, Wheelchair ac...",[Accepts reservations],"[Bar onsite, Good for kids, High chairs, Restr...",CA
362,Viviane Restaurant,"Viviane Restaurant, 9400 W Olympic Blvd, Bever...",0x80c2bbe33fe0d019:0xb942b62dff99b3ff,34.059113,-118.397197,"'New American restaurant', 'Restaurant'",4.4,92,$$$,"[Outdoor seating, No-contact delivery, Dine-in]",[Wheelchair accessible entrance],[Accepts reservations],"[Bar onsite, Restroom, Wi-Fi]",CA
363,Saudie's Restaurant,"Saudie's Restaurant, 6249 Old Winter Garden Rd...",0x88e77903fa4633a3:0x10d6f33cc1c24637,28.538465,-81.467056,'Haitian restaurant',4.2,178,$$,"[Takeout, Delivery]",[Wheelchair accessible entrance],[Usually a wait],[Good for kids],FL


In [49]:
datos.to_parquet('metadata_sitios_new.parquet')